In [ ]:
!pip install shap lime transformers torch torchvision

In [19]:
# Importing necessary libraries
import torch
from transformers import XLMRobertaForTokenClassification, DistilBertForTokenClassification
from transformers import XLMRobertaTokenizer, DistilBertTokenizer
import shap
from lime.lime_text import LimeTextExplainer
import numpy as np

In [20]:
# Load models and tokenizers
model_name_xlm = 'xlm-roberta-base'  # XLM-Roberta
model_name_distil = 'distilbert-base-multilingual-cased'  # DistilBERT

tokenizer_xlm = XLMRobertaTokenizer.from_pretrained(model_name_xlm)
tokenizer_distil = DistilBertTokenizer.from_pretrained(model_name_distil)

In [ ]:
# Load models
model_xlm = XLMRobertaForTokenClassification.from_pretrained(model_name_xlm)
model_distil = DistilBertForTokenClassification.from_pretrained(model_name_distil)

In [ ]:
# Set models to evaluation mode
model_xlm.eval()
model_distil.eval()

In [23]:
# Sample tokenized input data
sample_data = [
    "John Smith visited New York City.",
    "The river bank near the bank of America was flooded."
]

In [24]:
# Function to tokenize input for SHAP
def tokenize_for_shap(text):
    return tokenizer_xlm(text, return_tensors="pt", padding=True, truncation=True)

In [30]:
# SHAP Implementation
def explain_with_shap(model, text):
    # Tokenize input
    tokenized_data = tokenize_for_shap(text)

    # Define a function to make predictions
    def predict_fn(input_ids):
        input_ids = torch.tensor(input_ids)  # Ensure input_ids is a tensor
        with torch.no_grad():
            outputs = model(input_ids)
            logits = outputs.logits
            return torch.softmax(logits, dim=2).cpu().numpy()  # Get probabilities

    # Create explainer
    explainer = shap.KernelExplainer(predict_fn, tokenized_data['input_ids'].cpu().numpy())

    # Compute SHAP values for the tokenized input
    shap_values = explainer.shap_values(tokenized_data['input_ids'].cpu().numpy())

    # Plot SHAP summary for the first class (change index if necessary)
    feature_names = tokenizer_xlm.convert_ids_to_tokens(tokenized_data['input_ids'][0].tolist())
    shap.summary_plot(shap_values[0], feature_names=feature_names)

In [37]:
def explain_with_lime(model, text):
    explainer = LimeTextExplainer(class_names=["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-MISC", "I-MISC"])  # Adjust class names as necessary

    # Define a prediction function for LIME
    def predict_fn(texts):
        inputs = tokenizer_distil(texts, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(inputs['input_ids'].to(model.device))
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=2)  # Shape (batch_size, seq_length, num_classes)
            return probabilities.cpu().numpy()  # Return as numpy array

    # Generate LIME explanation
    explanation = explainer.explain_instance(text, predict_fn, num_features=10)

    # Store results for reporting
    lime_analysis = {
        'lime_exp': explanation,
        'text': text
    }

    return lime_analysis

In [ ]:
# Explain the models
print("SHAP Explanation for XLM-Roberta:")
explain_with_shap(model_xlm, sample_data[0])
print("LIME Explanation for DistilBERT:")
explain_with_lime(model_distil, sample_data[1])